# Supervised Fine-Tuning with SFTTrainer

This notebook demonstrates how to fine-tune the `HuggingFaceTB/SmolLM2-135M` model using the `SFTTrainer` from the `trl` library. The notebook cells run and will finetune the model. You can select your difficulty by trying out different datasets.

<div style='background-color: lightblue; padding: 10px; border-radius: 5px; margin-bottom: 20px; color:black'>
    <h2 style='margin: 0;color:blue'>Exercise: Fine-Tuning SmolLM2 with SFTTrainer</h2>
    <p>Take a dataset from the Hugging Face hub and finetune a model on it. </p> 
    <p><b>Difficulty Levels</b></p>
    <p>🐢 Use the `HuggingFaceTB/smoltalk` dataset</p>
    <p>🐕 Try out the `bigcode/the-stack-smol` dataset and finetune a code generation model on a specific subset `data/python`.</p>
    <p>🦁 Select a dataset that relates to a real world use case your interested in</p>
</div>

In [1]:
# Install the requirements in Google Colab
# !pip install transformers datasets trl huggingface_hub

# Authenticate to Hugging Face
from huggingface_hub import login
import yaml

# Load the config file
with open("/Users/ohnmarhtun/lab/smol-course/config.yaml", "r") as file:
    config = yaml.safe_load(file)

# Extract the token
hf_token = config.get("huggingface_hub", {}).get("token")

if hf_token:
    # Log in to Hugging Face Hub
    login(token=hf_token)
    print("Successfully logged in to Hugging Face!")
else:
    print("Access token not found in config.yaml!")

#login()

# for convenience you can create an environment variable containing your hub token as HF_TOKEN

Successfully logged in to Hugging Face!


In [2]:
# Import necessary libraries
from transformers import AutoModelForCausalLM, AutoTokenizer
from datasets import load_dataset
from trl import SFTConfig, SFTTrainer, setup_chat_format
import torch

device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps" if torch.backends.mps.is_available() else "cpu"
)

# Load the model and tokenizer
model_name = "HuggingFaceTB/SmolLM2-135M"
model = AutoModelForCausalLM.from_pretrained(
    pretrained_model_name_or_path=model_name
).to(device)
tokenizer = AutoTokenizer.from_pretrained(pretrained_model_name_or_path=model_name)

# Set up the chat format
model, tokenizer = setup_chat_format(model=model, tokenizer=tokenizer)

# Set our name for the finetune to be saved &/ uploaded to
finetune_name = "SmolLM2-FT-MyDataset"
finetune_tags = ["smol-course", "module_1"]

# Generate with the base model

Here we will try out the base model which does not have a chat template. 

In [3]:
# Let's test the base model before training
prompt = "Write a haiku about programming"

# Format with template
messages = [{"role": "user", "content": prompt}]
formatted_prompt = tokenizer.apply_chat_template(messages, tokenize=False)

# Generate response
inputs = tokenizer(formatted_prompt, return_tensors="pt").to(device)
outputs = model.generate(**inputs, max_new_tokens=100)
print("Before training:")
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

Before training:
user
Write a haiku about programming
Write a haiku about programming
Write a haiku about programming
Write a haiku about programming
Write a haiku about programming
Write a haiku about programming
Write a haiku about programming
Write a haiku about programming
Write a haiku about programming
Write a haiku about programming
Write a haiku about programming
Write a haiku about programming
Write a haiku about programming
Write a haiku about programming
Write a haiku about programming
Write a


## Dataset Preparation

We will load a sample dataset and format it for training. The dataset should be structured with input-output pairs, where each input is a prompt and the output is the expected response from the model.

**TRL will format input messages based on the model's chat templates.** They need to be represented as a list of dictionaries with the keys: `role` and `content`,.

In [4]:
# Load a sample dataset
from datasets import load_dataset

# TODO: define your dataset and config using the path and name parameters
ds = load_dataset(path="HuggingFaceTB/smoltalk", name="everyday-conversations")

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [5]:
# TODO: 🦁 If your dataset is not in a format that TRL can convert to the chat template, you will need to process it. 
# Refer to the [module](../chat_templates.md)
# Function to process dataset into TRL chat template format
# Import necessary libraries
from datasets import load_dataset, Dataset, load_from_disk
from datasets import DatasetDict

# Inspect the dataset structure
print("Dataset Structure:")
print(ds)

Dataset Structure:
DatasetDict({
    train: Dataset({
        features: ['full_topic', 'messages'],
        num_rows: 2260
    })
    test: Dataset({
        features: ['full_topic', 'messages'],
        num_rows: 119
    })
})


In [6]:
# Inspect the first training example
print("First Example in Training Split:")
print(ds["train"][0])  # Adjust 'train' if your dataset split is different

First Example in Training Split:
{'full_topic': 'Travel/Vacation destinations/Beach resorts', 'messages': [{'content': 'Hi there', 'role': 'user'}, {'content': 'Hello! How can I help you today?', 'role': 'assistant'}, {'content': "I'm looking for a beach resort for my next vacation. Can you recommend some popular ones?", 'role': 'user'}, {'content': "Some popular beach resorts include Maui in Hawaii, the Maldives, and the Bahamas. They're known for their beautiful beaches and crystal-clear waters.", 'role': 'assistant'}, {'content': 'That sounds great. Are there any resorts in the Caribbean that are good for families?', 'role': 'user'}, {'content': 'Yes, the Turks and Caicos Islands and Barbados are excellent choices for family-friendly resorts in the Caribbean. They offer a range of activities and amenities suitable for all ages.', 'role': 'assistant'}, {'content': "Okay, I'll look into those. Thanks for the recommendations!", 'role': 'user'}, {'content': "You're welcome. I hope you f

In [7]:
# Function to process dataset into TRL chat template format
def process_dataset_to_chat_template(dataset):
    """
    Processes the dataset to convert it into the TRL chat template format.

    Args:
        dataset: The dataset to process.
    Returns:
        A Dataset in the required chat template format.
    """
    chat_data = []

    # Iterate through each example in the dataset
    for example in dataset:
        # Access the 'messages' field
        messages = example.get("messages", [])

        # Initialize an empty chat template
        chat_template = ""

        # Iterate through messages and build the conversation template
        for msg in messages:
            role = msg.get("role", "")
            content = msg.get("content", "")
            
            if role and content:
                if role == "user":
                    chat_template += f"<|im_start|>user\n{content}<|im_end|>"
                elif role == "assistant":
                    chat_template += f"<|im_start|>assistant\n{content}<|im_end|>"

        # Append the chat template if not empty
        if chat_template:
            chat_data.append({"text": chat_template})

    if not chat_data:
        raise ValueError("No valid examples found in the dataset! Check the dataset structure and keys.")

    # Convert to Hugging Face Dataset
    return Dataset.from_list(chat_data)

# Process the dataset
if "train" in ds:
    processed_train = process_dataset_to_chat_template(ds["train"])
else:
    raise ValueError("Dataset does not have a 'train' split! Check the dataset structure.")

# Wrap the processed dataset into a DatasetDict
processed_dataset = DatasetDict({"train": processed_train})

if "test" in ds:
    processed_test = process_dataset_to_chat_template(ds["test"])
else:
    raise ValueError("Dataset does not have a 'test' split! Check the dataset structure.")

# Wrap the processed dataset into a DatasetDict
processed_dataset = DatasetDict({"train": processed_train,
                                "test": processed_test})

# Inspect the processed dataset
print("Sample Processed Dataset Entry:")
if len(processed_dataset["train"]) > 0:
    print(processed_dataset["train"][0])
else:
    print("Processed dataset is empty!")

# Save the processed dataset
processed_dataset.save_to_disk("processed_chat_template_dataset")

Sample Processed Dataset Entry:
{'text': "<|im_start|>user\nHi there<|im_end|><|im_start|>assistant\nHello! How can I help you today?<|im_end|><|im_start|>user\nI'm looking for a beach resort for my next vacation. Can you recommend some popular ones?<|im_end|><|im_start|>assistant\nSome popular beach resorts include Maui in Hawaii, the Maldives, and the Bahamas. They're known for their beautiful beaches and crystal-clear waters.<|im_end|><|im_start|>user\nThat sounds great. Are there any resorts in the Caribbean that are good for families?<|im_end|><|im_start|>assistant\nYes, the Turks and Caicos Islands and Barbados are excellent choices for family-friendly resorts in the Caribbean. They offer a range of activities and amenities suitable for all ages.<|im_end|><|im_start|>user\nOkay, I'll look into those. Thanks for the recommendations!<|im_end|><|im_start|>assistant\nYou're welcome. I hope you find the perfect resort for your vacation.<|im_end|>"}


Saving the dataset (0/1 shards):   0%|          | 0/2260 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/119 [00:00<?, ? examples/s]

In [8]:
# Load the processed dataset
dataset_path = "processed_chat_template_dataset"
processed_dataset = load_from_disk(dataset_path)
# Verify dataset splits
train_dataset = processed_dataset["train"]
eval_dataset = processed_dataset["test"]  # Ensure the dataset has a 'test' split


## Configuring the SFTTrainer

The `SFTTrainer` is configured with various parameters that control the training process. These include the number of training steps, batch size, learning rate, and evaluation strategy. Adjust these parameters based on your specific requirements and computational resources.

In [9]:
# Configure the SFTTrainer
finetune_name="SmolLM2-135M-FT2"

sft_config = SFTConfig(
    output_dir="./sft_output",
    max_steps=1000,  # Adjust based on dataset size and desired training duration
    per_device_train_batch_size=4,  # Set according to your GPU memory capacity
    learning_rate=5e-5,  # Common starting point for fine-tuning
    logging_steps=10,  # Frequency of logging training metrics
    save_steps=100,  # Frequency of saving model checkpoints
    evaluation_strategy="steps",  # Evaluate the model at regular intervals
    eval_steps=50,  # Frequency of evaluation
    use_mps_device=(
        True if device == "mps" else False
    ),  # Use MPS for mixed precision training
    hub_model_id=finetune_name,  # Set a unique name for your model
    max_seq_length=1024,
)


# Initialize the SFTTrainer
trainer = SFTTrainer(
    model=model,
    args=sft_config,
    train_dataset=train_dataset,
    tokenizer=tokenizer,
    eval_dataset=eval_dataset,
)

# TODO: 🦁 🐕 align the SFTTrainer params with your chosen dataset. For example, if you are using the `bigcode/the-stack-smol` dataset, you will need to choose the `content` column`






/Users/ohnmarhtun/anaconda3/envs/smol/lib/python3.11/site-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/Users/ohnmarhtun/anaconda3/envs/smol/lib/python3.11/site-packages/transformers/training_args.py:2248: UserWarning: `use_mps_device` is deprecated and will be removed in version 5.0 of 🤗 Transformers. `mps` device will be used by default if available similar to the way `cuda` device is used.Therefore, no action from user is required. 
  warnings.warn(
max_steps is given, it will override any value given in num_train_epochs


## Training the Model

With the trainer configured, we can now proceed to train the model. The training process will involve iterating over the dataset, computing the loss, and updating the model's parameters to minimize this loss.

In [10]:
# Train the model
trainer.train()

# Save the model
trainer.save_model(f"./{finetune_name}")

Step,Training Loss,Validation Loss
50,1.111900,1.209085
100,1.158200,1.174122
150,1.108600,1.143977
200,1.092300,1.127601
250,1.084500,1.118386
300,1.075500,1.108997
350,1.046000,1.101919
400,1.050100,1.097304
450,1.064800,1.088775
500,1.120800,1.079500


In [11]:
#trainer.push_to_hub(tags=finetune_tags)

<div style='background-color: lightblue; padding: 10px; border-radius: 5px; margin-bottom: 20px; color:black'>
    <h2 style='margin: 0;color:blue'>Bonus Exercise: Generate with fine-tuned model</h2>
    <p>🐕 Use the fine-tuned to model generate a response, just like with the base example..</p>
</div>

In [13]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

# Define the prompt
prompt = "Write a haiku about programming"

# Device setup
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps" if torch.backends.mps.is_available() else "cpu"
)

# Load the fine-tuned model and tokenizer
fine_tuned_model_path = "./SmolLM2-135M-FT2"  # Path where the fine-tuned model is saved
model = AutoModelForCausalLM.from_pretrained(fine_tuned_model_path).to(device)
tokenizer = AutoTokenizer.from_pretrained(fine_tuned_model_path)

# Ensure the pad token is set correctly
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token  # Set pad token to the end-of-sequence token if not defined

# Format the prompt with a chat template
messages = [{"role": "user", "content": prompt}]
formatted_prompt = tokenizer.apply_chat_template(messages, tokenize=False)

# Tokenize the prompt with padding and attention mask
inputs = tokenizer(
    formatted_prompt,
    return_tensors="pt",
    padding=True,  # Enable padding
    truncation=True,  # Ensure the input is not longer than the model's max length
    max_length=512,
).to(device)

# Generate response using the fine-tuned model
output = model.generate(
    inputs["input_ids"],
    attention_mask=inputs["attention_mask"],  # Explicitly set the attention mask
    max_length=512,  # Adjust based on desired response length
    temperature=0.9,  # Adjust for creativity (lower = more deterministic, higher = more creative)
    top_p=0.9,  # Nucleus sampling
    do_sample=True,  # Enable sampling for diverse outputs
)

# Decode the response
response = tokenizer.decode(output[0], skip_special_tokens=True)

# Print the fine-tuned model's response
print("Fine-tuned model response:")
print(response)

Fine-tuned model response:
user
Write a haiku about programming

What is a haiku about? A haiku is a Japanese poetic form that uses a 5-7-5 syllable pattern. It's often used in nature and abstract poetry.

What is a haiku about? A common type of haiku is the 5-7-5 pattern, which is used in nature, poetry, and many other forms. It's used to describe a flower, a leaf, or a simple word.

How can I write a haiku about programming? A simple way to write a haiku about programming is to focus on the 5-7-5 pattern. Use a simple word, phrase, or sentence that can describe a concept, idea, or action. This can help you create a more memorable and impactful haiku.

What is a haiku about? A simple way to create a more memorable and engaging haiku is to focus on the 5-7-5 pattern. Try to include a word or phrase that can describe a concept, idea, or action. This can make it more meaningful and memorable for the reader.

What is a haiku about? A simple way to create an engaging and memorable haiku is

## 💐 You're done!

This notebook provided a step-by-step guide to fine-tuning the `HuggingFaceTB/SmolLM2-135M` model using the `SFTTrainer`. By following these steps, you can adapt the model to perform specific tasks more effectively. If you want to carry on working on this course, here are steps you could try out:

- Try this notebook on a harder difficulty
- Review a colleagues PR
- Improve the course material via an Issue or PR.